# Add a new plugin

By default, the library will import all the files with prefix "plugin\_\*.py" from src/synthcity/plugins, and load all the classes which implement the [Plugin interface](src/synthcity/plugins/core/plugin.py).

Each plugin must implement the following methods:
- hyperparameter_space() - a static method that returns the hyperparameters that can be tuned during AutoML.
- type() - a static method that returns the type of the plugin. e.g., debug, generative, bayesian, etc.
- name() - a static method that returns the name of the plugin. e.g., ctgan, random_noisee, etc.
- _fit() - internal method, called by `fit` on each training set.
- _generate() - internal method, called by `generate`.

## Existing plugins

In [1]:
from synthcity.plugins import Plugins

generators = Plugins()

generators.list()

['marginal_distributions', 'random_noise']

## Example plugin: Generate 0-1

In [2]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class ZeroOnePlugin(Plugin):
    """Dummy plugin for debugging.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "zero_one"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: pd.DataFrame, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        self.features_count = X.shape[1]
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        return np.random.randint(0, 2, size=(count, self.features_count))

In [3]:
# Add the new plugin to the collection

generators.add("zero_one", ZeroOnePlugin)

In [4]:
# Check the new plugins list
generators.list()

['marginal_distributions', 'zero_one', 'random_noise']

In [5]:
# Load reference data

from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y = True, as_frame = True)

X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
# Train the new plugin

gen = generators.get("zero_one")

gen.fit(X)

In [7]:
# Generate some new data

gen.generate(count = 10)

RuntimeError: Plugin zero_one failed to meet the synthetic constraints.

### Oops, this didn't work.

__The Plugin interface enforces the new generated data to:__
 - satistify the same constraints as the training set.
 - Or to satisfy the constraints provided at inference time(if provided).
 
 
 If the generated dataframe fails to comply, an exception will be raised.

Let's try again

## A functional plugin

In [8]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class DummyGeneratorPlugin(Plugin):
    """Dummy plugin for debugging.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "dummy_generator"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: pd.DataFrame, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        result = self.schema().sample(count) 
        result[syn_schema.features()] = syn_schema.sample(count)
        
        return result

In [9]:
generators.add("dummy_generator", DummyGeneratorPlugin)

generators.list()

['dummy_generator', 'marginal_distributions', 'zero_one', 'random_noise']

In [10]:
# Train the new plugin

gen = generators.get("dummy_generator")

gen.fit(X)

In [11]:
# Generate some new data

gen.generate(count = 10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,12.252733,28.786770,65.320060,2187.526015,0.129633,0.306382,0.404331,0.163822,0.165654,0.059807,...,33.190514,20.431631,239.926898,3129.420384,0.104306,0.769573,0.489098,0.016853,0.229078,0.127259
1,23.747089,36.268859,116.916008,1564.332926,0.063677,0.159610,0.417602,0.037554,0.122535,0.077672,...,23.172315,24.260372,226.378601,2972.209662,0.096569,0.619530,1.207788,0.040610,0.526514,0.106626
2,18.506157,10.716171,151.028299,370.145378,0.071088,0.104190,0.086497,0.086636,0.215595,0.064866,...,15.230040,17.746544,217.927270,596.498417,0.136238,0.456739,0.107940,0.237724,0.283178,0.155933
3,21.796671,24.816409,66.441814,1962.212313,0.074983,0.230336,0.173910,0.132075,0.191789,0.060835,...,34.345572,17.045344,240.529627,2588.592260,0.155826,0.310510,0.028765,0.016835,0.291828,0.098895
4,18.035419,11.044487,53.827594,931.147885,0.102811,0.035941,0.363211,0.081962,0.206531,0.085614,...,26.593671,22.372981,118.694477,316.772343,0.081518,0.398965,0.806245,0.151890,0.584852,0.077786
5,17.595977,13.894407,151.157988,2231.712255,0.102699,0.222190,0.422657,0.107632,0.156390,0.095954,...,9.449882,15.959197,141.408313,4031.289820,0.159501,0.979498,0.076606,0.029667,0.569524,0.172235
6,20.205439,10.762855,164.512335,991.343833,0.092090,0.260358,0.181187,0.200636,0.268053,0.092159,...,26.909489,30.395979,183.129120,1989.422408,0.187898,0.559226,0.826083,0.099416,0.564112,0.173672
7,15.121121,30.086360,187.656147,390.186743,0.089068,0.037955,0.069143,0.047366,0.193472,0.061616,...,19.522075,35.164035,132.121672,1622.500990,0.190089,0.114198,0.418101,0.014080,0.544729,0.086303
8,23.726965,16.712532,166.295556,2269.850741,0.124782,0.091231,0.202501,0.132488,0.303580,0.089201,...,23.111654,28.232822,61.373770,3734.709668,0.154005,0.808079,0.274356,0.227688,0.650592,0.126233
9,13.953160,11.429358,173.465293,2264.401085,0.097628,0.097260,0.383056,0.197948,0.153270,0.075166,...,34.139528,40.672874,219.786619,3611.325551,0.216218,0.243632,0.576288,0.219685,0.409317,0.174318


In [12]:
# Custom generation constraints

from synthcity.plugins.core.constraints import Constraints

constraints = Constraints(rules = [("worst radius", ">", 15)])

generated = gen.generate(count = 10, constraints = constraints)

assert (generated["worst radius"] > 15).any()

generated

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,9.964698,19.637344,117.444782,1898.775532,0.074190,0.339937,0.116778,0.156297,0.109209,0.052263,...,8.950688e+08,40.577443,184.463155,2824.660136,0.162047,0.704618,0.361244,0.189326,0.590777,0.095254
1,10.749906,17.665361,104.156876,1141.894321,0.076007,0.107446,0.404464,0.106864,0.180705,0.078813,...,2.032342e+09,43.178418,99.446162,1820.996235,0.150713,0.678308,0.849608,0.196290,0.548413,0.132757
2,20.786793,33.640307,154.883371,2004.036697,0.156919,0.154534,0.317078,0.020577,0.230935,0.091240,...,1.130744e+09,12.741724,146.816846,4119.123937,0.194179,0.721399,0.156196,0.214986,0.450651,0.109817
3,7.405695,22.721220,64.087349,2085.617482,0.135968,0.031479,0.183181,0.172850,0.221023,0.078597,...,5.066379e+08,49.067970,213.671884,2070.647131,0.135199,0.322033,0.973152,0.019496,0.572471,0.076028
4,10.749036,37.870052,48.727987,423.540471,0.162776,0.039123,0.077182,0.177371,0.251939,0.067911,...,9.969882e+08,42.007719,111.212715,2144.030848,0.109642,1.007624,1.020547,0.071060,0.363551,0.154893
5,7.614535,19.704897,65.243283,1173.927061,0.129039,0.236335,0.300268,0.020945,0.289136,0.086127,...,2.143333e+08,15.431202,161.393316,2392.754987,0.152772,0.377740,1.200110,0.259138,0.642282,0.141444
6,15.285874,10.670832,81.074555,1245.205472,0.086573,0.267663,0.235283,0.038080,0.234888,0.086427,...,8.432324e+08,24.370803,184.871035,267.596643,0.153145,0.502817,0.435817,0.172866,0.476408,0.201045
7,18.628242,10.468127,110.185685,1243.159262,0.079488,0.194830,0.104103,0.161255,0.296369,0.090337,...,2.017041e+09,39.932972,190.264022,2198.595445,0.205264,0.125190,0.602469,0.138693,0.435965,0.073928
8,26.789477,14.750940,177.474016,618.016109,0.105214,0.091940,0.106389,0.152361,0.181259,0.058718,...,1.763572e+09,40.375099,64.112057,1648.686845,0.143573,0.601618,1.121840,0.128925,0.180798,0.115820
9,11.876695,15.525204,70.576273,1690.402885,0.158541,0.108098,0.188598,0.166591,0.194246,0.064767,...,5.790511e+08,41.213482,186.006298,3610.060022,0.205466,0.117939,0.520916,0.169194,0.311933,0.207316
